# **Revamping Hyperspectral Image Security with LAB color space Encryption**

**Libraries**

In [42]:
!pip install spectral
!pip install -U scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [43]:
from spectral import *
import scipy.io as sio
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import cv2
import pywt
from skimage.color import rgb2lab, lab2rgb
from sklearn.decomposition import PCA

from skimage.color import rgb2lab
from skimage import io, color, img_as_float

import os
from matplotlib.pyplot import imshow
import random
from math import log
from google.colab.patches import cv2_imshow

**Getting - Loading Dataset**

In [ ]:
!wget http://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
!wget http://www.ehu.eus/ccwintco/uploads/c/c4/Indian_pines_gt.mat

URL transformed to HTTPS due to an HSTS policy
--2023-04-18 06:05:23--  https://www.ehu.eus/ccwintco/uploads/6/67/Indian_pines_corrected.mat
Resolving www.ehu.eus (www.ehu.eus)... 158.227.0.65, 2001:720:1410::65
Connecting to www.ehu.eus (www.ehu.eus)|158.227.0.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5953527 (5.7M)
Saving to: ‘Indian_pines_corrected.mat.1’

        Indian_pine  66%[============>       ]   3.78M   195KB/s    eta 10s    

In [ ]:
# Load the hyperspectral data
data_file = 'Indian_pines_corrected.mat'
data = sio.loadmat(data_file)['indian_pines_corrected']

# Print the shape of the data
print('Data shape:', data.shape) # Third attribute is the total number of bands

In [ ]:
# Load the ground truth labels
labels_file = 'Indian_pines_gt.mat'
labels = sio.loadmat(labels_file)['indian_pines_gt']

# Print the shape of the labels
print('Labels shape:', labels.shape)

In [ ]:
# Display one of the bands and the color space name
band = 20
fig, ax = plt.subplots(1, 5, figsize=(12,5))
ax[0].imshow(data[:,:,band-20], cmap='Spectral')
ax[0].set_title('Band {}'.format(band-19))
ax[1].imshow(data[:,:,band], cmap='Spectral')
ax[1].set_title('Band {}'.format(band+1))
ax[2].imshow(data[:,:,band+50], cmap='Spectral')
ax[2].set_title('Band {}'.format(band+51))
ax[3].imshow(data[:,:,band+70], cmap='Spectral')
ax[3].set_title('Band {}'.format(band+71))
ax[4].imshow(data[:,:,band+100], cmap='Spectral')
ax[4].set_title('Band {}'.format(band+101))
plt.show()

In [ ]:
# Save one band as myband.png
img = data[:, :, band]
plt.imsave('myband.png', img, cmap='Spectral')

**Preprocess**

In [ ]:
# Preprocess the data
X = data.reshape(-1, data.shape[-1]) # reshape into (samples,features)
pca = PCA(n_components=30)
X_pca = pca.fit_transform(X)
X_norm = (X_pca - np.min(X_pca)) / (np.max(X_pca) - np.min(X_pca))
X_norm = X_norm.reshape(data.shape[:-1] + (X_norm.shape[-1],))

In [ ]:
# Print important features
eig_vecs = pca.components_
print('Top 3 eigenvectors:\n', eig_vecs[:3,:])

In [ ]:
from scipy import fftpack

# Load the image
img = plt.imread('myband.png')

# Convert the image to grayscale
gray_img = np.mean(img, axis=-1)

# Compute the 2D fast Fourier transform of the image
f = fftpack.fft2(gray_img)

# Create a Gaussian filter with standard deviation of 10
rows, cols = gray_img.shape
rs, cs = np.meshgrid(np.arange(rows), np.arange(cols), indexing='ij')
center_row, center_col = rows / 2, cols / 2
gaussian = np.exp(-((rs - center_row)**2 + (cs - center_col)**2) / (2 * (10**2)))

# Apply the filter to the Fourier spectrum of the image
smoothed_f = f * gaussian

# Compute the inverse Fourier transform to obtain the smoothed image
smoothed_img = np.real(fftpack.ifft2(smoothed_f))

# Normalize the pixel values to between 0 and 1
smoothed_img = (smoothed_img - np.min(smoothed_img)) / (np.max(smoothed_img) - np.min(smoothed_img))

# Show the original and smoothed images side by side
fig, ax = plt.subplots(1, 2, figsize=(7, 5))
ax[0].imshow(gray_img, cmap='Spectral')
ax[0].set_title('Original Image')
ax[1].imshow(smoothed_img, cmap='Spectral')
ax[1].set_title('Smoothed Image')
plt.show()

1. Convert HSI to LAB color space

In [ ]:
# Select a band from the hyperspectral data
band_num = 30
band = data[:, :, band_num]

# Convert the band to LAB space
#data_gray = np.stack([band_norm, band_norm, band_norm], axis=-1)
data_lab = color.rgb2lab(np.dstack((band, band, band)))

oneband = data[:,:,band_num]

# Convert the LAB image to RGB for display
data_rgb = color.lab2rgb(data_lab)

2. Split the LAB channels into L, a, and b

In [ ]:
L, a, b = cv2.split(data_lab)
fig, ax = plt.subplots(1, 4, figsize=(12,5))
ax[0].imshow(L, cmap='Spectral')
plt.imsave('l.png', a, cmap='Spectral')
ax[0].set_title('L channel')

ax[1].imshow(a, cmap='Spectral')
plt.imsave('a.png', a, cmap='Spectral')
ax[1].set_title('a channel')

ax[2].imshow(b, cmap='Spectral')
plt.imsave('b.png', a, cmap='Spectral')
ax[2].set_title('b channel')

ax[3].imshow(oneband, cmap='Spectral')
plt.imsave('lab.png', a, cmap='Spectral')
ax[3].set_title('Overall band')
plt.show()

3. Encryption and Decryption algorithm : Pseudorandom Chaotic Image Scrambling (PCIS)


In [ ]:
def pcis_encrypt_lab(img, keys):
    # Convert the image to LAB color space
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)

    # Initialize the pseudorandom number generator for each color channel
    rng_l, rng_a, rng_b = [np.random.default_rng(key) for key in keys]

    # Create a chaotic permutation matrix for each color channel
    perm_l, perm_a, perm_b = [np.zeros_like(img_lab[:,:,0]) for i in range(3)]
    for i in range(5):
        perm_l = np.mod(np.power(perm_l, 2.1) + i, 255)
        perm_a = np.mod(np.power(perm_a, 2.1) + i + 1, 255)
        perm_b = np.mod(np.power(perm_b, 2.1) + i + 2, 255)
    perm_l = perm_l.ravel().argsort().reshape(perm_l.shape)
    perm_a = perm_a.ravel().argsort().reshape(perm_a.shape)
    perm_b = perm_b.ravel().argsort().reshape(perm_b.shape)

    # Generate a random mask and scramble the image using the mask and permutation matrix
    mask_l = rng_l.integers(0, 256, size=img_lab.shape[:2], dtype=np.uint8)
    mask_a = rng_a.integers(0, 256, size=img_lab.shape[:2], dtype=np.uint8)
    mask_b = rng_b.integers(0, 256, size=img_lab.shape[:2], dtype=np.uint8)
    img_scrambled = cv2.merge([
        cv2.bitwise_xor(img_lab[:,:,0], mask_l), 
        cv2.bitwise_xor(img_lab[:,:,1], mask_a), 
        cv2.bitwise_xor(img_lab[:,:,2], mask_b)
    ])
    img_scrambled[:,:,0] = np.take(img_scrambled[:,:,0], perm_l.ravel()).reshape(img_lab[:,:,0].shape)
    img_scrambled[:,:,1] = np.take(img_scrambled[:,:,1], perm_a.ravel()).reshape(img_lab[:,:,1].shape)
    img_scrambled[:,:,2] = np.take(img_scrambled[:,:,2], perm_b.ravel()).reshape(img_lab[:,:,2].shape)

    # Return the encrypted image and the keys
    return img_scrambled, keys

4. Results

In [ ]:
import cryptography
import time

# Generate random keys for each color channel
img = cv2.imread("lab.png")

key_l = np.random.bytes(16)
key_a = np.random.bytes(16)
key_b = np.random.bytes(16)
keys = [x for x in bytearray(b'\xef\xc0K\x1dhx"\x1c\x95\x12\xcd\xdf\xaf\x86\xaa#')]

# Encrypt the image
start_time = time.time()

img_encrypted, key_encrypted = pcis_encrypt_lab(img, [keys]*3)

end_time = time.time()
encryption_time = end_time - start_time

# Calculate the length of the key in bytes and bits
key_length_bytes = len(keys)
key_length_bits = key_length_bytes * 8

print(f"Key length (bits): {key_length_bits}")
print(f"Encryption time: {encryption_time} seconds")

In [ ]:
# Compute the mean squared error between the original and decrypted images
'''
mse = np.mean((img - img_decrypted) ** 2)
'''
fig, ax = plt.subplots(1, 2, figsize=(8,5))
ax[0].imshow(img, cmap='Spectral')
ax[0].set_title('Original')

ax[1].imshow(img_encrypted, cmap='Spectral')
ax[1].set_title('Encrypted')
'''
ax[2].imshow(img_decrypted, cmap='Spectral')
ax[2].set_title('Decrypted')
'''

plt.show()

In [ ]:
import math

# Calculate the number of possible keys
num_possible_keys = 2 ** key_length_bits

# Calculate the number of operations required for a brute-force attack
num_operations = num_possible_keys / 2

# Calculate the time required for a brute-force attack
num_operations_per_second = 1e9  # Assume 1 billion operations per second
time_to_brute_force = num_operations / num_operations_per_second

# Print the results
print(f"Key size: {key_length_bits} bits ({key_length_bytes} bytes)")
print(f"Number of possible keys: {num_possible_keys}")
print(f"Number of operations for brute-force attack: {num_operations}")
print(f"Time required for brute-force attack: {time_to_brute_force:.2f} seconds")